Imports

In [ ]:
import os
import numpy as np
import chess
import chess.pgn
import chess.engine
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input

Const and hiperparameters

In [ ]:
LEARNING_RATE = 0.001
STOCKFISH_PATH = 'stockfish/stockfish-windows-x86-64.exe'

PGN to position

In [ ]:
def parse_pgn_files(pgn_directory):
    positions = []
    for filename in os.listdir(pgn_directory):
        if filename.endswith(".pgn"):
            with open(os.path.join(pgn_directory, filename), 'r', encoding='utf-8') as pgn_file:
                while True:
                    game = chess.pgn.read_game(pgn_file)
                    if game is None:
                        break
                    board = game.board()
                    for move in game.mainline_moves():
                        board.push(move)
                        fen = board.fen()
                        positions.append(fen)

    return positions

Position to input

In [ ]:
def state_to_input(board):
    planes = np.zeros((8, 8, 12))
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_type = piece.piece_type - 1  # Typ figury od 0 do 5
            color = 0 if piece.color == chess.WHITE else 6  # Białe: 0-5, Czarne: 6-11
            row = 7 - chess.square_rank(square)
            col = chess.square_file(square)
            planes[row, col, piece_type + color] = 1
    return planes

Engine evaluation

In [ ]:
def get_engine_evaluation(board, stockfish_path=STOCKFISH_PATH):
    with chess.engine.SimpleEngine.popen_uci(stockfish_path) as engine:
        info = engine.analyse(board, chess.engine.Limit(depth=10))
        score = info["score"].white().score(mate_score=10000)
        engine.quit()
    return score / 100.0  

Neural nertwork

In [ ]:
def build_pretrained_model(input_shape=(8, 8, 12)):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='linear'))
    optimizer = tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

Preparation

In [ ]:
positions = parse_pgn_files('pgn_files/')

X = []
y = []

In [ ]:
for idx, fen in enumerate(positions):
    board = chess.Board(fen)
    input_state = state_to_input(board)
    X.append(input_state)
    evaluation = get_engine_evaluation(board)
    y.append(evaluation)
    if (idx + 1) % 100 == 0:
    print(f'Przetworzono {idx + 1}/{len(positions)} pozycji')


X = np.array(X)
y = np.array(y)

Training

In [ ]:
model = build_pretrained_model()
model.fit(X, y, epochs=10, batch_size=64, validation_split=0.1)

model.save('pretrained_model.h5')